##  Let me try Transformer architecture!!

- 20 Oct 2020

In [1]:
!git clone https://github.com/fastai/course-v3/ /content/course-v3
%cd /content/course-v3/nbs/dl2

Cloning into '/content/course-v3'...
remote: Enumerating objects: 5893, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5893 (delta 0), reused 2 (delta 0), pack-reused 5890
Receiving objects: 100% (5893/5893), 263.10 MiB | 38.77 MiB/s, done.
Resolving deltas: 100% (3251/3251), done.
/content/course-v3/nbs/dl2


- NOTE: if you do not have installed [nvidia/apex](https://github.com/nvidia/apex),  change `from exp.nb_10c import *` to `from exp.nb_10b import *` in `exp.nb_11`

In [2]:
from exp.nb_12a import *

- Local data (which is divided into separate dataset) is saved to `Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/v1/ser'`


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
root_path = Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1'); root_path.ls()

[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/dev')]

features-entry contains a list of a list with 26 items.
- length of inner list: 26 (float numbers - represent one preprocessed speech frame (logMel))

- length of outer list: number of frames per data-point, e.g. 10 or 15, ...

In [5]:
train_path = root_path/'train'

In [6]:
audios = get_files(train_path)

In [7]:
# restart-run
class AudioList(ItemList):
    @classmethod
    def from_files(cls, path, extensions = None, recurse=True, include=None, **kwargs):
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, fn):
        return torch.load(fn)

In [8]:
al=AudioList.from_files(train_path); al

AudioList (7800 items)
[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7760_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7761_0_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7762_0_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7763_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7764_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7765_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7766_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7767_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1/train/7768_1_0.p

## Use adaptive pooling to fit the frames

## input transform

1. load tensor
2. tocuda
3. transpose, add dimension to first axis
4. AdaptiveAvgPooling to fit width(time frames) 250


In [9]:
class ToCuda():
    _order=10
    def __call__(self, ad):
        return ad.cuda()

In [10]:
class Reshape():
    _order=11
    def __call__(self, item):
        w, h = item.shape
        return item.view(h, w)

In [ ]:
class DummyFeature():
    # add features from function
    _order= 13
    def __init__(self, f, **kwargs):
        self.p = partial(f, **kwargs)
    def __call__(self, item):
        return torch.vstack((item, self.p(item).unsqueeze(0)))

In [11]:
# Mutants of input tensor
class PadorTrim():
    _order = 20
    def __init__(self, max_len):
        self.max_len = max_len
    def __call__(self, ad):
        # h - logmel, here 27, w - frames / various
        h, w = ad.shape
        pad_size = self.max_len - w
        if pad_size >0: return torch.cat((ad, torch.zeros(h, pad_size).cuda()), dim=1)
        else: return ad[:, :self.max_len]

class TimeFramePooling():
    _order= 12
    def __init__(self, f, n_tf):
        # n_tf: numner of time frames
        self.p = f(n_tf)
    def __call__(self, item):
        return self.p(item)

# TimeFramePooling(f= nn.AdaptiveMaxPool1d, n_tf = 150)

class DummyChannel():
    _order = 30
    def __call__(self, item):
        return item.unsqueeze(0)

In [15]:
tfms = [ToCuda(), Reshape(), PadorTrim(225), DummyChannel()]
compose(al[0], tfms).shape

torch.Size([1, 26, 225])

In [16]:
il = AudioList.from_files(train_path, tfms = tfms)

- This scaling process is annoying, because I should have this process 'before' training, while separating train and eval status

In [17]:
il[0].shape

torch.Size([1, 26, 225])

In [18]:
def re_labeler(fn, pat, subcl='act'):
    assert subcl in ['act', 'val', 'all']
    if subcl=='all': return ''.join(re.findall(pat, str(fn)))
    else:
        return re.findall(pat, str(fn))[0] if pat == 'act' else re.findall(pat, str(fn))[1]

label_pat = r'_(\d+)'
emotion_labeler = partial(re_labeler, pat=label_pat, subcl='all')

In [19]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.2))
ll = label_by_func(sd, emotion_labeler, proc_y=CategoryProcessor())

In [20]:
bs=64
c_in = ll.train[0][0].shape[0]
c_out = len(uniqueify(ll.train.y))
data = ll.to_databunch(bs,c_in=c_in,c_out=c_out)

In [21]:
c_in, c_out

(1, 4)

In [28]:
import inspect
def show_resource(x): print(inspect.getsource(x))
show_resource(XResNet)

class XResNet(nn.Sequential):
    @classmethod
    def create(cls, expansion, layers, c_in=3, c_out=1000):
        nfs = [c_in, (c_in+1)*8, 64, 64]
        stem = [conv_layer(nfs[i], nfs[i+1], stride=2 if i==0 else 1)
            for i in range(3)]

        nfs = [64//expansion,64,128,256,512]
        res_layers = [cls._make_layer(expansion, nfs[i], nfs[i+1],
                                      n_blocks=l, stride=1 if i==0 else 2)
                  for i,l in enumerate(layers)]
        res = cls(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *res_layers,
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(nfs[-1]*expansion, c_out),
        )
        init_cnn(res)
        return res

    @staticmethod
    def _make_layer(expansion, ni, nf, n_blocks, stride):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
              for i in range(n_blocks)])



In [30]:
def get_encoder():
    return nn.TransformerEncoderLayer(
            d_model=26,
            nhead=2,
            dim_feedforward=256,
            dropout=0.4,
            activation='gelu')

In [32]:
def reshape(x): return x.permute(2,0,1)

In [34]:
class Reshape(nn.Module):
    def __init__(self): super().__init__()
    def forward(self,x):
        return x.squeeze(1).permute(2, 0 ,1)

In [48]:
class FC(nn.Module):
    def __init__(self):
        super().__init__()
        self.f= nn.Linear(26, 4)
    def forward(self,x):
        return self.f(x.mean(dim=0))

In [49]:
def get_model():
    return nn.Sequential(
        nn.MaxPool2d(kernel_size=[1,4], stride=[1,4]),
        Reshape(),
        nn.TransformerEncoder(get_encoder(), 3),
        FC()
    )

In [52]:
xb, yb = next(iter(data.train_dl))
xb.shape, yb.shape

(torch.Size([64, 1, 26, 225]), torch.Size([64]))

In [50]:
model = get_model().cuda()

In [51]:
model

Sequential(
  (0): MaxPool2d(kernel_size=[1, 4], stride=[1, 4], padding=0, dilation=1, ceil_mode=False)
  (1): Reshape()
  (2): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=26, out_features=26, bias=True)
        )
        (linear1): Linear(in_features=26, out_features=256, bias=True)
        (dropout): Dropout(p=0.4, inplace=False)
        (linear2): Linear(in_features=256, out_features=26, bias=True)
        (norm1): LayerNorm((26,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((26,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.4, inplace=False)
        (dropout2): Dropout(p=0.4, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=26, out_features=26, bias=True)
        )
        (linear1)

In [53]:
model(xb).shape

torch.Size([64, 4])

In [61]:
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)
loss_func = LabelSmoothingCrossEntropy()
lr = 1e-2
pct_start = 0.5
phases = create_phases(pct_start)
sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds(phases, cos_1cycle_anneal(0.95,0.85, 0.95))
cbscheds = [ParamScheduler('lr', sched_lr), 
            ParamScheduler('mom', sched_mom)]

In [56]:
!pip install -qq torchmetrics
from torchmetrics import F1
f1 = F1(num_classes=4)

In [64]:
f1.__name__ = 'fscore'

In [ ]:
#@title
class Learner():
    def __init__(self, model, data, loss_func, opt_func=sgd_opt, lr=1e-2, splitter=param_getter,
                 cbs=None, cb_funcs=None):
        self.model,self.data,self.loss_func,self.opt_func,self.lr,self.splitter = model,data,loss_func,opt_func,lr,splitter
        self.in_train,self.logger,self.opt = False,print,None

        # NB: Things marked "NEW" are covered in lesson 12
        # NEW: avoid need for set_runner
        self.cbs = []
        self.add_cb(TrainEvalCallback())
        self.add_cbs(cbs)
        self.add_cbs(cbf() for cbf in listify(cb_funcs))

    def add_cbs(self, cbs):
        for cb in listify(cbs): self.add_cb(cb)

    def add_cb(self, cb):
        cb.set_runner(self)
        setattr(self, cb.name, cb)
        self.cbs.append(cb)

    def remove_cbs(self, cbs):
        for cb in listify(cbs): self.cbs.remove(cb)

    def one_batch(self, i, xb, yb):
        try:
            self.iter = i
            self.xb,self.yb = xb,yb;                        self('begin_batch')
            self.pred = self.model(self.xb);                self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb); self('after_loss')
            if not self.in_train: return
            self.loss.backward();                           self('after_backward')
            self.opt.step();                                self('after_step')
            self.opt.zero_grad()
        except CancelBatchException:                        self('after_cancel_batch')
        finally:                                            self('after_batch')

    def all_batches(self):
        self.iters = len(self.dl)
        try:
            for i,(xb,yb) in enumerate(self.dl): self.one_batch(i, xb, yb)
        except CancelEpochException: self('after_cancel_epoch')

    def do_begin_fit(self, epochs):
        self.epochs,self.loss = epochs,tensor(0.)
        self('begin_fit')

    def do_begin_epoch(self, epoch):
        self.epoch,self.dl = epoch,self.data.train_dl
        return self('begin_epoch')

    def fit(self, epochs, cbs=None, reset_opt=False):
        # NEW: pass callbacks to fit() and have them removed when done
        self.add_cbs(cbs)
        # NEW: create optimizer on fit(), optionally replacing existing
        if reset_opt or not self.opt: self.opt = self.opt_func(self.splitter(self.model), lr=self.lr)

        try:
            self.do_begin_fit(epochs)
            for epoch in range(epochs):
                self.do_begin_epoch(epoch)
                if not self('begin_epoch'): self.all_batches()

                with torch.no_grad():
                    self.dl = self.data.valid_dl
                    if not self('begin_validate'): self.all_batches()
                self('after_epoch')

        except CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.remove_cbs(cbs)

    ALL_CBS = {'begin_batch', 'after_pred', 'after_loss', 'after_backward', 'after_step',
        'after_cancel_batch', 'after_batch', 'after_cancel_epoch', 'begin_fit',
        'begin_epoch', 'begin_epoch', 'begin_validate', 'after_epoch',
        'after_cancel_train', 'after_fit'}

    def __call__(self, cb_name):
        res = False
        assert cb_name in self.ALL_CBS
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) and res
        return res


In [70]:
def learner(arch, data, loss_func, opt_func, c_in=None, c_out=None,
                lr=1e-2, cuda=True, norm=None, progress=True, mixup=0, xtra_cb=None, **kwargs):
    cbfs = [partial(AvgStatsCallback,accuracy)]+listify(xtra_cb)
    if progress: cbfs.append(ProgressCallback)
    if cuda:     cbfs.append(CudaCallback)
    if norm:     cbfs.append(partial(BatchTransformXCallback, norm))
    if mixup:    cbfs.append(partial(MixUp, mixup))
    arch_args = {}
    if not c_in : c_in  = data.c_in
    if not c_out: c_out = data.c_out
    if c_in:  arch_args['c_in' ]=c_in
    if c_out: arch_args['c_out']=c_out
    return Learner(arch, data, loss_func, opt_func=opt_func, lr=lr, cb_funcs=cbfs, **kwargs)


In [71]:
learn = learner(model, data, loss_func, opt_func=opt_func)

In [72]:
show_resource(accuracy)

def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()



In [73]:
learn.fit(300, cbs=cbscheds)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.292441,0.401639,1.304084,0.402408,37:16
1,1.265722,0.427997,1.295530,0.403676,00:10
2,1.243031,0.447605,1.479739,0.304816,00:10
3,1.227606,0.452588,1.530696,0.301014,00:10
4,1.216821,0.458534,1.639924,0.224335,00:10
5,1.207194,0.464481,1.520950,0.288973,00:10
6,1.207225,0.466410,1.394439,0.339037,00:10
7,1.198821,0.478464,1.709912,0.254119,00:10
8,1.198704,0.476696,1.722170,0.201521,00:10
9,1.195064,0.477821,1.540053,0.266793,00:10


/content/course-v3/nbs/dl2/exp/nb_09.py:136: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  state['grad_avg'].mul_(mom).add_(state['mom_damp'], p.grad.data)


KeyboardInterrupt: ignored

In [79]:
def save_checkpoint(learner, filename):
    fname = Path(filename)
    fname.parent.mkdir(parents=True, exist_ok = True)
    checkpoint_dict = {
        'model': learn.model.state_dict(),
    }
    torch.save(checkpoint_dict, fname)

In [80]:
save_checkpoint(learn, Path('/content/2021-12-20-v2-2-transformer.pt'))

# References

- [Pytorch Transformer Encoder](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html)
- [fastai course-v3 part2, audio](https://github.com/fastai/course-v3/blob/master/nbs/dl2/audio.ipynb)
- [Transformer-CNN-emotion recognition](https://github.com/IliaZenkov/transformer-cnn-emotion-recognition/blob/main/notebooks/Parallel_is_All_You_Want.ipynb)
- 
